# Two Sample t-test
Chaitanya Anand <br>
(https://github.com/decisive-moment) <br>
Created on: Jan 04, 2020 <br>
Last Edit: March 03, 2020 <br>
Notebook to demonstrate two sample t-test using R. <br> Deliberately not used R's inbuilt t.test function in order to explore the inner workings of the test. <br><br>
This notebook assumes that the reader is familiar with the theory of hypothesis testing. If not then I would recommend reading up on the basics before going through the code. <br><br>
Some resources that I have found useful for learning hypothesis testing are: <br>(By no means an exhaustive list. One may pick up any resource on introductry statistics) <br>
- Design and Analysis of Experiments by Montgomery ISBN-13: 978-1118146927
- Coursera course on inferential statistics: https://www.coursera.org/learn/inferential-statistics/home/welcome

In [ ]:
# Loading important libraries
library(dplyr)
library(ggplot2)

### Important Formulas

Degrees of freedom for the equal variance case<br>
$
\begin{align}
df = n_1 + n_2 - 2 \\
\end{align}
$<br>

Degrees of freedom for the unequal variance case<br>
$
\begin{align}
df = \frac{\left(\frac{s_1^2}{n_1}+\frac{s_2^2}{n_2}\right)^2}{\frac{\left(\frac{s_1^2}{n_1}\right)^2}{n_1-1} + \frac{\left(\frac{s_2^2}{n_2}\right)^2}{n_2-1}}\\
\end{align}
$<br>

Standard deviation for the equal variance case<br>
$
\begin{align}
s_{\bar{X_1}-\bar{X_2}}=\sqrt{\frac{(n_1-1)s_1^2-(n_2-1)s_2^2}{n_1+n_2-2}}\\
\end{align}
$<br>

Standard deviation for the unequal variance case<br>
$
\begin{align}
s_{\bar{X_1}-\bar{X_2}}=\sqrt{s_1^2+s_2^2}
\end{align}
$

Standard error for the equal variance case<br>
$
\begin{align}
SE_{\bar{X_1}-\bar{X_2}}=\sqrt{\left(\frac{(n_1-1)s_1^2-(n_2-1)s_2^2}{n_1+n_2-2}\right)\left(\frac{1}{n_1}+\frac{1}{n_2}\right)}\\
\end{align}
$<br>

Standard error for the unequal variance case<br>
$
\begin{align}
SE_{\bar{X_1}-\bar{X_2}}=\sqrt{\frac{s_1^2}{n_1}+\frac{s_2^2}{n_2}}
\end{align}
$

### Data Generation Process

In [ ]:
#### -------- Parameters of the experiment
# Number of trials from the first random process
n1 = 100
# Number of trials from the second random process
n2 = 100
# Chosen significance level
alpha = 0.05
# Assume equal variance of the two samples?
eqvar = FALSE
# Two tailed or one tailed test?
twotail = TRUE

#### -------- Population parameters
# Actual population mean of the first random process
mu1 = 11
# Actual population mean of the second random process
mu2 = 12
# Actual population standard deviation of the first random process
sigma1 = 3
# Actual population standard deviation of the econd random process
sigma2 = 2

In [ ]:
#### -------- Let us say have collected some data from the two random processes
# The rnorm function in r helps us simulate exactly this
# The function will generate 'n' random values drawn from a
# normal distribution of the specified parameters (mu and sigma)

treatment1_data <- rnorm(n = n1, mean = mu1, sd = sigma1)
treatment2_data <- rnorm(n = n2, mean = mu2, sd = sigma2)

# We will keep the true difference in a variable for calculating power of the test later on
true_difference = mu1 - mu2

In [ ]:
# We shall treat the population parameters as unknowm
# throughout the hypothesis testing process
# their only purpose was to obtain the simulated data
# in the previous step

# Deleting them from memory
rm(mu1, mu2, sigma1, sigma2)

### Stating the  Hypothesis
Let us say we have collected some data from two treatments (variations) of a random process. <br>
e.g. Measuring the height of plants where one group of seedlings is grown in clay soil (treatment 1) and another group is grown in loamy soil (treatment 2). We now want to detect if clay soil (treatment 1) and loamy soil (treatment 2) differ in their effect on plant growth.

The null hypothesis in this case would be:
\begin{align}
H_0: \mu_1 - \mu_2 = 0
\end{align}

The alternative hypothesis can be stated as (assuming we have no reason to believe one type of soil is better than the other):
\begin{align}
H_1: \mu_1 - \mu_2 \neq 0
\end{align}

### Calculating Sample Statistics

In [189]:
# Data collected from random process 1
glimpse(treatment1_data)

 num [1:100] 8.47 14.81 12.32 15.22 11.58 ...


In [190]:
# Data collected from random process 2
glimpse(treatment2_data)

 num [1:100] 10.6 11.7 10.9 13.7 13.1 ...


In [ ]:
# Sample mean of data collected from process 1
x1_bar = mean(treatment1_data)
# Sample mean of data collected from process 2
x2_bar = mean(treatment2_data)

# Sample standard deviation of data collected from process 1
s1 = sd(treatment1_data)
# Sample standard deviation of data collected from process 2
s2 = sd(treatment2_data)

# Estimate of the population standard deviation
# Obtained using the sample standard deviations
# Since we are now treating the actual population parameters as unknown
# (please see the section on important formulas at the begining of this document)

var = ifelse(eqvar,

              # Equal standard deviation case
              # (i.e. assuming that the two process have the same standard deviation)
              (((((n1 -1)*(s1^2)) + ((n2 -1)*(s2^2)))/(n1+n2-2))),

              # Unequal standard deviation case
              # (i.e. assuming that the two process have different standard deviation)
              (s1^2) + (s2^2)
)
sd = var^(1/2)


# Standard error
# (please see the section on important formulas at the begining of this document)
se = ifelse(eqvar,

              # Equal standard deviation case
              # (i.e. assuming that the two process have the same standard deviation)
              (((((n1 -1)*(s1^2)) + ((n2 -1)*(s2^2)))/(n1+n2-2))*((1/n1) + (1/n2)))^(1/2),

              # Unequal standard deviation case
              # (i.e. assuming that the two process have different standard deviation)
              (((s1^2)/n1) + ((s2^2)/n2))^(1/2)
)


# Degrees of freedom
# (please see the section on important formulas at the begining of this document)
# Equal standard deviation case
df = ifelse(eqvar,

              # Equal standard deviation case
              # (i.e. assuming that the two process have the same standard deviation)
              n1 + n2 - 2,

              # Unequal standard deviation case
              # (i.e. assuming that the two process have different standard deviation)
              (((s1^2/n1) + (s2^2/n2))^2)/
                (((s1^2/n1)^2/(n1-1)) + (((s2^2/n2)^2)/(n2-1)))
)

# Observed effect (diffference between the two treatments i.e. random processes)
observed_dfference = x1_bar - x2_bar
# z value of the observed effect
# Since null hypothesis is that the difference in means is zero
# The sampling distribution is centered at mu = 0
z = (0 - observed_dfference)/se

In [192]:
x1_bar
x2_bar
x1_bar-x2_bar

[1] 10.81994

[1] 11.87217

[1] -1.052228

In [193]:
s1
s2

[1] 3.228988

[1] 1.965014

In [194]:
se
z

[1] 0.37799

[1] 2.783746

### Step by Steb Procedure

H0: Treatment 1 and Treatment 2 are not diferent i.e. mu1 = mu2 or mu1 - mu2 = 0 <br>
H1: Treatment 1 and Treatment 2 are the same i.e. mu1 != mu2 or mu1 - mu2 != 0 <br>

In [195]:
p = pt(q = z, df = df, lower.tail = ifelse(observed_dfference > 0, TRUE, FALSE), log.p = FALSE)
  p = ifelse(twotail,
              p*2,
              p
  )
format(p, scientific = TRUE)
format(round(p, 5), scientific = FALSE)

[1] "6.00715e-03"

[1] "0.00601"

In [196]:
ifelse(p < alpha, "We reject the null hypothesis", "We fail to reject the null hypothesis")

[1] "We reject the null hypothesis"

### Alternative Approaoch - Confidence Interval

In [ ]:
critical_value = qt(p = ifelse(twotail, alpha/2, alpha), df = df, lower.tail = TRUE, log.p = FALSE)
lower_limit = ifelse(sign(critical_value) == 1 & sign (observed_dfference) == 1,
                 observed_dfference - (critical_value*se),
                 observed_dfference + (critical_value*se))
upper_limit = ifelse(sign(critical_value) == 1 & sign (observed_dfference) == 1,
                 observed_dfference + (critical_value*se),
                 observed_dfference - (critical_value*se))
ci = c(lower_limit, observed_dfference, upper_limit)


power = 1-ifelse(twotail,
    
                    pt(q = ifelse(true_difference > 0,
                                  (abs(critical_value*se) - true_difference)/se,
                                  (abs(true_difference) - abs(critical_value*se))/se),
                       df = df,
                       lower.tail = ifelse(true_difference > 0, TRUE, FALSE),
                       log.p = FALSE) -
                    pt(q = ifelse(true_difference > 0,
                                  (- abs(true_difference) - abs(critical_value*se))/se,
                                  (abs(true_difference) + abs(critical_value*se))/se),
                       df = df,
                       lower.tail = ifelse(true_difference > 0, TRUE, FALSE), log.p = FALSE),

                    pt(q = ifelse(true_difference > 0,
                                  (abs(critical_value*se) - true_difference)/se,
                                  (abs(true_difference) -  abs(critical_value*se))/se),
                       df = df, lower.tail = ifelse(true_difference > 0, TRUE, FALSE),
                       log.p = FALSE))
    

In [198]:
ci

[1] -1.7986003 -1.0522284 -0.3058564

In [199]:
power

[1] 0.7484174

In [200]:
ifelse(0 >= ci[1] & 0 <= ci[3],
       "The confidence interval contains zero",
       "The confidence interval (corresponding to chosen alpha) does not contain 0")

[1] "The confidence interval (corresponding to chosen alpha) does not contain 0"

### Verifying the results using R's inbuilt t.test function

In [201]:
t.test(treatment1_data,
       treatment2_data,
       mu = 0,
       paired = FALSE,
       var.equal = eqvar,
       alternative = ifelse(twotail, "two.sided", "one.sided"),
       conf.level = 1 - alpha)


	Welch Two Sample t-test

data:  treatment1_data and treatment2_data
t = -2.7837, df = 163.48, p-value = 0.006007
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -1.7986003 -0.3058564
sample estimates:
mean of x mean of y 
 10.81994  11.87217 


### Encapsulating the above process in a function (to enable iterative experimentation)

In [ ]:
two_sample_t <- function(n1, n2, alpha, mu1, mu2, sigma1, sigma2, eqvar, twotail, validate = FALSE) {
    
    true_difference = mu1 - mu2
    
    # Generating data
    treatment1_data <- rnorm(n = n1, mean = mu1, sd = sigma1)
    treatment2_data <- rnorm(n = n2, mean = mu2, sd = sigma2)
    

    # Sample mean of data collected from process 1
    x1_bar = mean(treatment1_data)
    # Sample mean of data collected from process 2
    x2_bar = mean(treatment2_data)

    # Sample standard deviation of data collected from process 1
    s1 = sd(treatment1_data)
    # Sample standard deviation of data collected from process 2
    s2 = sd(treatment2_data)

    # Estimate of the population standard deviation
    # Obtained using the sample standard deviations
    # Since we are now treating the actual population parameters as unknown
    # (please see the section on important formulas at the begining of this document)

    var = ifelse(eqvar,

                  # Equal standard deviation case
                  # (i.e. assuming that the two process have the same standard deviation)
                  (((((n1 -1)*(s1^2)) + ((n2 -1)*(s2^2)))/(n1+n2-2))),

                  # Unequal standard deviation case
                  # (i.e. assuming that the two process have different standard deviation)
                  (s1^2) + (s2^2)
    )
    sd = var^(1/2)


    # Standard error
    # (please see the section on important formulas at the begining of this document)
    se = ifelse(eqvar,

                  # Equal standard deviation case
                  # (i.e. assuming that the two process have the same standard deviation)
                  (((((n1 -1)*(s1^2)) + ((n2 -1)*(s2^2)))/(n1+n2-2))*((1/n1) + (1/n2)))^(1/2),

                  # Unequal standard deviation case
                  # (i.e. assuming that the two process have different standard deviation)
                  (((s1^2)/n1) + ((s2^2)/n2))^(1/2)
    )


    # Degrees of freedom
    # (please see the section on important formulas at the begining of this document)
    # Equal standard deviation case
    df = ifelse(eqvar,

                  # Equal standard deviation case
                  # (i.e. assuming that the two process have the same standard deviation)
                  n1 + n2 - 2,

                  # Unequal standard deviation case
                  # (i.e. assuming that the two process have different standard deviation)
                  (((s1^2/n1) + (s2^2/n2))^2)/
                    (((s1^2/n1)^2/(n1-1)) + (((s2^2/n2)^2)/(n2-1)))
    )

    # Observed effect (diffference between the two treatments i.e. random processes)
    observed_dfference = x1_bar - x2_bar
    # z value of the observed effect
    # Since null hypothesis is that the difference in means is zero
    # The sampling distribution is centered at mu = 0
    z = (0 - observed_dfference)/se












    # Calculating p-value
    p = pt(q = z, df = df, lower.tail = ifelse(observed_dfference > 0, TRUE, FALSE), log.p = FALSE)
    p = ifelse(twotail,
                p*2,
                p
    )
    # format(p, scientific = TRUE)
    # format(round(p, 5), scientific = FALSE)
    
    # Calculating the cnfidence interval
    critical_value = qt(p = ifelse(twotail, alpha/2, alpha), df = df, lower.tail = TRUE, log.p = FALSE)
    lower_limit = ifelse(sign(critical_value) == 1 & sign (observed_dfference) == 1,
                    observed_dfference - (critical_value*se),
                    observed_dfference + (critical_value*se))
    upper_limit = ifelse(sign(critical_value) == 1 & sign (observed_dfference) == 1,
                    observed_dfference + (critical_value*se),
                    observed_dfference - (critical_value*se))
    ci = c(lower_limit, observed_dfference, upper_limit)

    power = 1 - ifelse(twotail,
    
                    pt(q = ifelse(true_difference > 0,
                                  (abs(critical_value*se) - true_difference)/se,
                                  (abs(true_difference) - abs(critical_value*se))/se),
                       df = df,
                       lower.tail = ifelse(true_difference > 0, TRUE, FALSE),
                       log.p = FALSE) -
                    pt(q = ifelse(true_difference > 0,
                                  (- abs(true_difference) - abs(critical_value*se))/se,
                                  (abs(true_difference) + abs(critical_value*se))/se),
                       df = df,
                       lower.tail = ifelse(true_difference > 0, TRUE, FALSE), log.p = FALSE),

                    pt(q = ifelse(true_difference > 0,
                                  (abs(critical_value*se) - true_difference)/se,
                                  (abs(true_difference) -  abs(critical_value*se))/se),
                       df = df, lower.tail = ifelse(true_difference > 0, TRUE, FALSE),
                       log.p = FALSE))
    
    # Results of the hypothesis test
    null_hypothesis_test_method <- ifelse(p <= alpha, "We reject the null hypothesis", "We fail to reject the null hypothesis")
    confidence_interval_method <- ifelse(ci[1] <= 0 & 0 <= ci[3],
       "The confidence interval contains zero",
       "The confidence interval (corresponding to chosen alpha) does not contain 0")
    
    ret <- list(ifelse(null_hypothesis_test_method == "We reject the null hypothesis" &
                  confidence_interval_method == "The confidence interval (corresponding to chosen alpha) does not contain 0", "Reject",
                  
                  ifelse(null_hypothesis_test_method == "We fail to reject the null hypothesis" &
                         confidence_interval_method == "The confidence interval contains zero", "Fail to reject",
                         "ERROR")),
                         power = power,
                p = p,
                ci = ci,
                x1_bar = x1_bar,
                x2_bar = x2_bar,
                s1 = s1,
                s2 = s2,
                se = se)

    # Validatoin
    if(validate){
          print(t.test(treatment1_data,
          treatment2_data,
          mu = 0,
          paired = FALSE,
          var.equal = eqvar,
          alternative = ifelse(twotail, "two.sided", ifelse(mu1 > mu2, "greater", "less")),
          conf.level = 1 - alpha))
    }
    
    return(ret)
}

In [ ]:
two_sample_t_power <- function(alpha, n1, n2, actual_effect_size, sigma1, sigma2, eqvar, twotail) {
        se = (((sigma1^2)/n1) + ((sigma2^2)/n2))^(1/2)
        df = ifelse(eqvar,

                          # Equal standard deviation case
                          # (i.e. assuming that the two process have the same standard deviation)
                          n1 + n2 - 2,

                          # Unequal standard deviation case
                          # (i.e. assuming that the two process have different standard deviation)
                          (((sigma1^2/n1) + (sigma2^2/n2))^2)/
                            (((sigma1^2/n1)^2/(n1-1)) + (((sigma1^2/n2)^2)/(n2-1)))
            )
        critical_value = qt(p = ifelse(twotail, alpha/2, alpha), df = df, lower.tail = ifelse(actual_effect_size > 0, TRUE, FALSE), log.p = FALSE)
        power = 1 - ifelse(twotail,
            
                            pt(q = ifelse(actual_effect_size > 0,
                                          (abs(critical_value*se) - actual_effect_size)/se,
                                          (abs(actual_effect_size) - abs(critical_value*se))/se),
                               df = df,
                               lower.tail = ifelse(actual_effect_size > 0, TRUE, FALSE), log.p = FALSE) -
                            pt(q = ifelse(actual_effect_size > 0,
                                         ( - abs(actual_effect_size) - abs(critical_value*se))/se,
                                          (abs(actual_effect_size) + abs(critical_value*se))/se),
                               df = df, lower.tail = ifelse(actual_effect_size > 0, TRUE, FALSE), log.p = FALSE),

                            pt(q = ifelse(actual_effect_size > 0,
                                          (abs(critical_value*se) - actual_effect_size)/se,
                                          (abs(actual_effect_size) - abs(critical_value*se))/se),
                               df = df, lower.tail = ifelse(actual_effect_size > 0, TRUE, FALSE), log.p = FALSE))

        return(power)
}

In [204]:
# Parameters of the experiment
exp1_n1 = 100
exp1_n2 = 100
exp1_alpha = 0.05
exp1_eqvar = FALSE
exp1_twotail = FALSE

# Population parameters

exp1_mu1 = 11
exp1_mu2 = 14
exp1_sigma1 = 3
exp1_sigma2 = 3




print(paste0("power = ", two_sample_t_power(alpha = exp1_alpha,
                                            n1 = exp1_n1,
                                            n2 = exp1_n2,
                                            actual_effect_size = exp1_mu1 - exp1_mu2,
                                            sigma1 = exp1_sigma1,
                                            sigma2 = exp1_sigma2,
                                            eqvar = exp1_eqvar,
                                            twotail = exp1_twotail)))

two_sample_t(n1 = exp1_n1,
             n2 = exp1_n2,
             alpha = exp1_alpha,
             mu1 = exp1_mu1,
             mu2 = exp1_mu2,
             sigma1 = exp1_sigma1,
             sigma2 = exp1_sigma2,
             eqvar = exp1_eqvar,
             twotail = exp1_twotail,
             validate = TRUE)

[1] "power = 0.999999913285151"

	Welch Two Sample t-test

data:  treatment1_data and treatment2_data
t = -9.1184, df = 197.66, p-value < 2.2e-16
alternative hypothesis: true difference in means is less than 0
95 percent confidence interval:
      -Inf -3.072152
sample estimates:
mean of x mean of y 
 10.79571  14.54791 



[[1]]
[1] "Reject"

$power
[1] 1

$p
[1] 4.380436e-17

$ci
[1] -4.432235 -3.752193 -3.072152

$x1_bar
[1] 10.79571

$x2_bar
[1] 14.54791

$s1
[1] 2.969568

$s2
[1] 2.848637

$se
[1] 0.4114981

In [205]:
# Iterate

# Parameters of the experiment
exp1_n1 = 100
exp1_n2 = 100
exp1_alpha = 0.05
exp1_eqvar = FALSE
exp1_twotail = TRUE

# Population parameters

exp1_mu1 = 12
exp1_mu2 = 12
exp1_sigma1 = 3
exp1_sigma2 = 3

print(paste0("power = ", two_sample_t_power(alpha = exp1_alpha,
                                            n1 = exp1_n1,
                                            n2 = exp1_n2,
                                            actual_effect_size = exp1_mu1 - exp1_mu2,
                                            sigma1 = exp1_sigma1,
                                            sigma2 = exp1_sigma2,
                                            eqvar = exp1_eqvar,
                                            twotail = exp1_twotail)))

results <- c()
for(i in 1:100){
    res = two_sample_t(n1 = exp1_n1,
                       n2 = exp1_n2,
                       alpha = exp1_alpha,
                       mu1 = exp1_mu1,
                       mu2 = exp1_mu2,
                       sigma1 = exp1_sigma1,
                       sigma2 = exp1_sigma2,
                       eqvar = exp1_eqvar,
                       twotail = exp1_twotail)
    results[i] = res[[1]]
    # ifelse(results[i] == "Reject", print(res),NA)
}
table(results)

# Expected frequency of type 1 error = 5%
# i.e. probability of rejecting the null hypothesis when it is actually true

[1] "power = 0.0500000000000003"


results
Fail to reject         Reject 
            96              4 

In [216]:
# Iterate

# Parameters of the experiment
exp1_n1 = 1000000
exp1_n2 = 1000000
exp1_alpha = 0.05
exp1_eqvar = TRUE
exp1_twotail = FALSE

# Population parameters

exp1_mu1 = 11
exp1_mu2 = 11
exp1_sigma1 = 3
exp1_sigma2 = 3

print(paste0("power = ", two_sample_t_power(alpha = exp1_alpha,
                                            n1 = exp1_n1,
                                            n2 = exp1_n2,
                                            actual_effect_size = exp1_mu1 - exp1_mu2,
                                            sigma1 = exp1_sigma1,
                                            sigma2 = exp1_sigma2,
                                            eqvar = exp1_eqvar,
                                            twotail = exp1_twotail)))



results <- c()
for(i in 1:100){
    res = two_sample_t(n1 = exp1_n1,
                       n2 = exp1_n2,
                       alpha = exp1_alpha,
                       mu1 = exp1_mu1,
                       mu2 = exp1_mu2,
                       sigma1 = exp1_sigma1,
                       sigma2 = exp1_sigma2,
                       eqvar = exp1_eqvar,
                       twotail = exp1_twotail)
    results[i] = res[[1]]
    # ifelse(results[i] == "Reject", print(res),NA)
}
table(results)

# Expected frequency of type I error = 5%
# i.e. probability of rejecting the null hypothesis when it is actually true

[1] "power = 0.0499999999999999"


results
Fail to reject         Reject 
            89             11 

In [212]:
# Iterate

# Parameters of the experiment
exp1_n1 = 1000000
exp1_n2 = 1000000
exp1_alpha = 0.05
exp1_eqvar = TRUE
exp1_twotail = FALSE

# Population parameters

exp1_mu1 = 15
exp1_mu2 = 11
exp1_sigma1 = 3
exp1_sigma2 = 3

print(paste0("power = ", two_sample_t_power(alpha = exp1_alpha,
                                            n1 = exp1_n1,
                                            n2 = exp1_n2,
                                            actual_effect_size = exp1_mu1 - exp1_mu2,
                                            sigma1 = exp1_sigma1,
                                            sigma2 = exp1_sigma2,
                                            eqvar = exp1_eqvar,
                                            twotail = exp1_twotail)))



results <- c()
for(i in 1:100){
    res = two_sample_t(n1 = exp1_n1,
                       n2 = exp1_n2,
                       alpha = exp1_alpha,
                       mu1 = exp1_mu1,
                       mu2 = exp1_mu2,
                       sigma1 = exp1_sigma1,
                       sigma2 = exp1_sigma2,
                       eqvar = exp1_eqvar,
                       twotail = exp1_twotail)
    results[i] = res[[1]]
    # ifelse(results[i] == "Reject", print(res),NA)
}
table(results)

# Expected frequency of type II error  = 0% (since power = 1)
# i.e. probability of failing to reject the null hypothesis when it is actually false

[1] "power = 1"


results
Reject 
   100 

In [213]:
# Iterate

# Parameters of the experiment
exp1_n1 = 100
exp1_n2 = 100
exp1_alpha = 0.05
exp1_eqvar = TRUE
exp1_twotail = FALSE

# Population parameters

exp1_mu1 = 12
exp1_mu2 = 11
exp1_sigma1 = 3
exp1_sigma2 = 3


print(paste0("power = ", two_sample_t_power(alpha = exp1_alpha,
                                            n1 = exp1_n1,
                                            n2 = exp1_n2,
                                            actual_effect_size = exp1_mu1 - exp1_mu2,
                                            sigma1 = exp1_sigma1,
                                            sigma2 = exp1_sigma2,
                                            eqvar = exp1_eqvar,
                                            twotail = exp1_twotail)))


results <- c()
for(i in 1:100){
    res = two_sample_t(n1 = exp1_n1,
                       n2 = exp1_n2,
                       alpha = exp1_alpha,
                       mu1 = exp1_mu1,
                       mu2 = exp1_mu2,
                       sigma1 = exp1_sigma1,
                       sigma2 = exp1_sigma2,
                       eqvar = exp1_eqvar,
                       twotail = exp1_twotail)
    results[i] = res[[1]]
    # ifelse(results[i] == "Reject", print(res),NA)
}
table(results)

# Expected frequency of type II error  = 25% (since power = 1)
# i.e. probability of failing to reject the null hypothesis when it is actually false

[1] "power = 0.759005750597752"


results
Fail to reject         Reject 
            26             74 

In [217]:
# Iterate

# Parameters of the experiment
exp1_n1 = 1000000
exp1_n2 = 1000000
exp1_alpha = 0.05
exp1_eqvar = TRUE
exp1_twotail = FALSE

# Population parameters

exp1_mu1 = 11
exp1_mu2 = 11
exp1_sigma1 = 3
exp1_sigma2 = 3


print(paste0("power = ", two_sample_t_power(alpha = exp1_alpha,
                                            n1 = exp1_n1,
                                            n2 = exp1_n2,
                                            actual_effect_size = exp1_mu1 - exp1_mu2,
                                            sigma1 = exp1_sigma1,
                                            sigma2 = exp1_sigma2,
                                            eqvar = exp1_eqvar,
                                            twotail = exp1_twotail)))

results <- c()
for(i in 1:100){
    res = two_sample_t(n1 = exp1_n1,
                       n2 = exp1_n2,
                       alpha = exp1_alpha,
                       mu1 = exp1_mu1,
                       mu2 = exp1_mu2,
                       sigma1 = exp1_sigma1,
                       sigma2 = exp1_sigma2,
                       eqvar = exp1_eqvar,
                       twotail = exp1_twotail)
    results[i] = res[[1]]
    # ifelse(results[i] == "Reject", print(res),NA)
}
table(results)


# Expected frequency of type I error = 5%
# i.e. probability of rejecting the null hypothesis when it is actually true

[1] "power = 0.0499999999999999"


results
Fail to reject         Reject 
            90             10 